In [ ]:
import Toggel_utils as toggel
import omero_utils as omero

In [ ]:
import numpy as np
from pathlib import Path

### upload Frequency Domain FLIM data to OMERO

In [ ]:
# connect to OMERO
conn = omero.connect("rensham")

In [ ]:
# define path to folder containing raw .fli files
dir_path = r'C:\Users\rensham\OneDrive - The Francis Crick Institute\repositories\temp_files'

In [ ]:
# process all .fli files in dir_path

folder = Path(dir_path) # string to Path

project_name = "Toggel" # store all Toggel phase series within same 'Toggel' project
dataset_name = folder.stem

project_object = omero.create_project(conn, project_name) # get or create project called project_name
project_id = project_object.getId()

dataset_object = omero.create_dataset(conn, dataset_name, project_id) # get or create dataset called dataset_name within project_id
dataset_id = dataset_object.getId()

for file in folder.glob("*.fli"):
    filename = file.stem
    acquisition_datetime = filename[0:19]
    print(filename)

    # read .fli file and get metadata    
    md, phase_images, dark_image = toggel.read_fli_file(file)
    sizeX, sizeY, n_phases, mod_frequency = (md['LAYOUT'].get(key) for key in ('x', 'y', 'phases', 'modulationFrequency'))
    sensor_gain, ref = (md['ACQUISITION SETTINGS'].get(key) for key in ('sensorGain', 'RefLifetime'))
    
    if float(ref) > 0:
        acquisition_type = "reference"
    else:
        acquisition_type = "sample"

    key_value_pairs = []
    key_value_pairs.append(["Imaging Modality", "Frequency Domain FLIM"])
    key_value_pairs.append(["acquisition_datetime", str(acquisition_datetime)])
    key_value_pairs.append(["n_phases", str(n_phases)])
    key_value_pairs.append(["modulation_frequency_Hz", str(mod_frequency)])
    key_value_pairs.append(["sensor_gain", str(sensor_gain)])
    key_value_pairs.append(["reference_lifetime_ns", str(ref)])
    key_value_pairs.append(["acquisition_type", str(acquisition_type)])

    # subtract camera dark image from phase images
    phase_series = np.empty_like(phase_images, dtype=np.float32)
    for ph, plane in enumerate(phase_images):
        phase_series[ph] = plane.astype(np.float32) - dark_image.astype(np.float32)

    # upload images to OMERO
    image_obj = omero.create_image(conn, filename, dataset_id, key_value_pairs, phase_series, channel_names=None,
                             sizeT=int(n_phases), 
                             description="FD FLIM phase images")
    

### define path

In [ ]:
path_to_file = 'N:\\outputs\\treismanr\\fromMatt_Toggel\\20250902\\raw\\2025-09-02_13-21-43-sample-coumarin_ref_4-3ns.fli'

In [ ]:
path_to_file = r'C:\Users\rensham\OneDrive - The Francis Crick Institute\repositories\temp_files\2025-09-05_15-06-19-reference-20x_coumarin_4-3ns.fli'

In [ ]:
path_to_file = r'C:\Users\rensham\OneDrive - The Francis Crick Institute\repositories\temp_files\2025-09-05_15-10-01-sample-20x_purified_proteins.fli'

### read raw .fli file

In [ ]:
file = Path(path_to_file)
filename = file.stem
acquisition_datetime = filename[0:19]
print(filename)

In [ ]:
md, phase_images, dark_image = toggel.read_fli_file(path_to_file)

### extract metadata

In [ ]:
sizeX, sizeY, n_phases, mod_frequency = (md['LAYOUT'].get(key) for key in ('x', 'y', 'phases', 'modulationFrequency'))
sensor_gain, ref = (md['ACQUISITION SETTINGS'].get(key) for key in ('sensorGain', 'RefLifetime'))

In [ ]:
sizeX, sizeY, n_phases, mod_frequency

In [ ]:
sensor_gain, ref

In [ ]:
md

In [ ]:
md['TIMESTAMPS']['t0']

### subtract dark image from phase series

In [ ]:
# convert to 32-bit float
# subtract dark image from image phase series
phase_series = np.empty_like(phase_images, dtype=np.float32)
for ph, plane in enumerate(phase_images):
    phase_series[ph] = plane.astype(np.float32) - dark_image.astype(np.float32)

In [ ]:
from omero.gateway import MapAnnotationWrapper
from omero.constants.metadata import NSCLIENTMAPANNOTATION
from omero.model.enums import UnitsLength
from omero.model import LengthI
from omero.model import DatasetI, ProjectI, ProjectDatasetLinkI
from omero.rtypes import rstring, rlong, robject

def create_project(conn, project_name):
    """ 
    Create omero project.
    Checks if a project with project_name exists.
    
    
    Parameters:
        conn: Connected BlitzGateway.
        project_name (str): project name.
    
    Returns:
        OMERO project object.
    
    """
    # Check is project exists
    project_exists = None
    for proj in conn.getObjects("Project"):
        if proj.getName() == str(project_name):
            project_exists = proj
            break

    if project_exists:
        print(f"Project already exists: ID = {project_exists.getId()}, Name = {project_exists.getName()}")
        project_obj = project_exists
        project_id = project_obj.getId()
        project_name = project_obj.getName()

    else:
        # Create a new project
        project_obj = ProjectI()
        project_obj.setName(rstring(project_name))
        project_obj = conn.getUpdateService().saveAndReturnObject(project_obj)
        project_id = project_obj.getId().getValue()
        project_name = project_obj.getName().getValue()
        print(f"Created new project: ID = {project_id}, Name = {project_name}")

    # Re-load project object
    project_obj = conn.getObject("Project", project_id)
    return project_obj

In [ ]:
project_obj = create_project(conn, "Toggel")

In [ ]:
def create_dataset(conn, dataset_name, project_id):
    """ 
    Create omero dataset within defined project.
    Checks if a dataset with dataset_name exists.
    Creates and returns new dataset object or returns existing.
    
    Parameters:
        conn: Connected BlitzGateway.
        dataset_name (str): dataset name.
        project_id (int): 
        
    Returns:
        OMERO dataset object.
    
    """
    # Search for existing project with the same name
    dataset_exists = None
    project_obj = conn.getObject("Project", project_id)
    if project_obj.countChildren() > 0:
        for dataset in project_obj.listChildren():
            if dataset.getName() == str(dataset_name):
                dataset_exists = dataset
                break 

    if dataset_exists:
        print(f"Dataset already exists: ID = {dataset_exists.getId()}, Name = {dataset_exists.getName()}")
        dataset_obj = dataset_exists
        dataset_id = dataset_obj.getId()
        dataset_name = dataset_obj.getName()

    else:
        # Create a new dataset
        dataset_obj = DatasetI()
        dataset_obj.setName(rstring(dataset_name))
        dataset_obj = conn.getUpdateService().saveAndReturnObject(dataset_obj)
        dataset_id = dataset_obj.getId().getValue()
        dataset_name = dataset_obj.getName().getValue()
        print(f"Created new dataset: ID = {dataset_id}, Name = {dataset_name}")

        # link dataset to project
        link = ProjectDatasetLinkI()
        link.setParent(ProjectI(project_obj.getId(), False))
        link.setChild(dataset_obj)
        conn.getUpdateService().saveObject(link)

    # Re-load dataset object
    dataset_obj = conn.getObject("Dataset", dataset_id)
    return dataset_obj

In [ ]:
project_id = project_obj.getId()
project_id

In [ ]:
dataset_object = create_dataset(conn, "Test_Create_Dataset", project_id)

In [ ]:
if float(ref) > 0:
    acquisition_type = "reference"
else:
    acquisition_type = "sample"

In [ ]:
key_value_pairs = []
key_value_pairs.append(["Imaging Modality", "Frequency Domain FLIM"])
key_value_pairs.append(["acquisition_datetime", str(acquisition_datetime)])
key_value_pairs.append(["n_phases", str(n_phases)])
key_value_pairs.append(["modulation_frequency_Hz", str(mod_frequency)])
key_value_pairs.append(["sensor_gain", str(sensor_gain)])
key_value_pairs.append(["reference_lifetime_ns", str(ref)])
key_value_pairs.append(["acquisition_type", str(acquisition_type)])

In [ ]:
key_value_pairs

In [ ]:
def create_image(conn, image_name, dataset_id, key_value_pairs, image_planes, channel_names=None, description=None, 
                 sizeZ=1, sizeC=1, sizeT=1, pixel_size_um = 0, 
                 sourceImageId=None, channelList=None):
    """ 
    Create omero image within defined dataset.
    Checks existing image with image_name.
    Creates and returns new image object or returns existing.
    
    Parameters:
        conn: Connected BlitzGateway.
        image_name (str): name of new image.
        dataset_id (int): link image to this dataset
        key_value_pairs (list): List of metadata [key, value] pairs for MapAnnotations
        image_planes (numpy_ndarray): image planes in numpy sequence
        description (str): description for new image
        
    Returns:
        OMERO image object.
    
    """
    # Search for existing project with the same name
    image_exists = None
    dataset_obj = conn.getObject("Dataset", dataset_id)
    if dataset_obj.countChildren() > 0:
        for image in dataset_obj.listChildren():
            if image.getName() == str(image_name):
                image_exists = image
                break 

    if image_exists:
        print(f"Image already exists: ID = {image_exists.getId()}, Name = {image_exists.getName()}")
        image_obj = image_exists
        image_id = image_obj.getId()
        image_name = image_obj.getName()

    else:
        # Stack into a 3D array
        image_stack = np.stack(image_planes, axis=0)
        
        # Create a new image
        def plane_gen():
            """generator will yield planes"""
            for plane in image_stack:
                yield plane
                
        img_obj = conn.createImageFromNumpySeq(
                plane_gen(), image_name, sizeZ, sizeC, sizeT, description=description, 
                dataset=dataset_obj, sourceImageId=sourceImageId, channelList=channelList
            )
        
        image_id = img_obj.getId().getValue()
        image_name = img_obj.getName().getValue()
        print(f"Created new image: ID = {image_id}, Name = {image_name}")

        # set channel names
        channel_labels = dict(enumerate(channel_names, start=1))
        conn.setChannelNames('Image', [img_obj.getId()], 
                             channel_labels
                            )
        
        # set pixel sizes
        if pixel_size_um <= 0:
            px_size_um = LengthI(1, UnitsLength.PIXEL) # non calibrated pixel sizes
        else :
            px_size_um = LengthI(pixel_size_um, UnitsLength.MICROMETER)
        px_obj = img_obj.getPrimaryPixels()._obj
        px_obj.setPhysicalSizeX(px_size_um)
        px_obj.setPhysicalSizeY(px_size_um)
        conn.getUpdateService().saveObject(px_obj)

        # Re-load the image to avoid update conflicts
        img_obj = conn.getObject("Image", img_obj.getId())
        
        map_ann = MapAnnotationWrapper(conn)
        map_ann.setNs(NSCLIENTMAPANNOTATION)
        map_ann.setValue(key_value_pairs)
        map_ann.save()
        
        # NB: only link a client map annotation to a single object
        map_ann = img_obj.linkAnnotation(map_ann)

    # Re-load image object to avoid conflicts
    image_obj = conn.getObject("Image", image_id)
    
    return image_obj

In [ ]:
dataset_id = dataset_object.getId()
dataset_id

In [ ]:
image_obj = create_image(conn, filename, dataset_id, key_value_pairs, phase_series, channel_names=None,
                         sizeT=int(n_phases), 
                         description="FD FLIM phase images")